In [1]:
'''
This file takes S&P500 data (both the individual stocks and index) from yahoo finance and 
calculates the market return, market standard deviation, 
stock return, stock standard deviation, as well as stock return for the past 12 months less the recent 1/2/3/6 months.
It also calculates the dollar value (dv) (volume * closing price) change of each stock over the rolling 1/2/3/6/12 months.
Finally, it also calculates the % of positive returns over the past 1/2/3/6/12 months.
It then combines all the dataframes into a single dataframe, with dates and tickers as the rows and features as the columns.

df contains raw data downloaded from yfinance
df1 contains close data for all S&P 500 stocks and the index
daily_return_df contains the daily return data for all S&P 500 stocks and the index
M1_return_df contains the monthly return data for all S&P 500 stocks and the index
M2_return_df contains the 2 month return data for all S&P 500 stocks and the index
M3_return_df contains the 3 month return data for all S&P 500 stocks and the index
M6_return_df contains the 6 month return data for all S&P 500 stocks and the index
M12_return_df contains the 12 month return data for all S&P 500 stocks and the index

M1_forward_return_df contains the 1 month forward return data for all S&P 500 stocks and the index

M1_sd_df contains the 1 month standard deviation data for all S&P 500 stocks and the index
M2_sd_df contains the 2 month standard deviation data for all S&P 500 stocks and the index
M3_sd_df contains the 3 month standard deviation data for all S&P 500 stocks and the index
M6_sd_df contains the 6 month standard deviation data for all S&P 500 stocks and the index
M12_sd_df contains the 12 month standard deviation data for all S&P 500 stocks and the index

M12_1_return_df contains the 12 month return data minus 1 month return data for all S&P 500 stocks and the index
M12_2_return_df contains the 12 month return data minus 2 month return data for all S&P 500 stocks and the index
M12_3_return_df contains the 12 month return data minus 3 month return data for all S&P 500 stocks and the index
M12_6_return_df contains the 12 month return data minus 6 month return data for all S&P 500 stocks and the index

dv_df contains all the dollar value for all S&P 500 stocks and the index
dv_change_df contains the daily change in dv
M1_dv_change_df contains the monthly change in dv
M2_dv_change_df contains the 2 month change in dv
M3_dv_change_df contains the 3 month change in dv
M6_dv_change_df contains the 6 month change in dv
M12_dv_change_df contains the 12 month change in dv

M1_positive_returns_df contains the percentage of positive returns over past 1 month
M2_positive_returns_df contains the percentage of positive returns over past 2 months
M3_positive_returns_df contains the percentage of positive returns over past 3 months
M6_positive_returns_df contains the percentage of positive returns over past 6 months
M12_positive_returns_df contains the percentage of positive returns over past 12 months
'''

'\nThis file takes S&P500 data (both the individual stocks and index) from yahoo finance and \ncalculates the market return, market standard deviation, \nstock return, stock standard deviation, as well as stock return for the past 12 months less the recent 1/2/3/6 months.\nIt also calculates the dollar value (dv) (volume * closing price) change of each stock over the rolling 1/2/3/6/12 months.\nFinally, it also calculates the % of positive returns over the past 1/2/3/6/12 months.\nIt then combines all the dataframes into a single dataframe, with dates and tickers as the rows and features as the columns.\n\ndf contains raw data downloaded from yfinance\ndf1 contains close data for all S&P 500 stocks and the index\ndaily_return_df contains the daily return data for all S&P 500 stocks and the index\nM1_return_df contains the monthly return data for all S&P 500 stocks and the index\nM2_return_df contains the 2 month return data for all S&P 500 stocks and the index\nM3_return_df contains th

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pycaret

# Yahoo finance
import yfinance as yf

# Datetime
import datetime as dt
from datetime import datetime

In [3]:
start=pd.to_datetime('1990-01-01', format='%Y-%m-%d')
today_date = datetime.today().strftime('%Y-%m-%d')
period=256           #lookback period for calculation of beta, realized returns
period_1m=21
period_2m=42
period_3m=63
period_6m=128
period_12m=256


In [4]:
#Download data for multiple indicators available on Yahoo Finance

# Read and print the stock tickers that make up S&P500
tickers = pd.read_html(
    'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['Symbol']
tickers=tickers.to_list()

print(tickers)
#print(tickers.head())

['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BK', 'BBWI', 'BAX', 'BDX', 'BRK.B', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BX', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'BLDR', 'BG', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'COR', 'CNC', 'CNP', 'CF', 'CHRW', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CPAY', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CCI', 'CSX', 'CMI', 'C

In [5]:
#Two of the tickers have issue, also adding in S&P 500 index
for i in range(len(tickers)):
 
    # replace BF.B with BF-B
    if tickers[i] == 'BF.B':
        tickers[i] = 'BF-B'
 
    # replace BRK.B with BRK-B
    if tickers[i] == 'BRK.B':
        tickers[i] = 'BRK-B'

#Add S&P index
tickers.append("^GSPC")
# print list
print(tickers)

['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BK', 'BBWI', 'BAX', 'BDX', 'BRK-B', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BX', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF-B', 'BLDR', 'BG', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'COR', 'CNC', 'CNP', 'CF', 'CHRW', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CPAY', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CCI', 'CSX', 'CMI', 'C

In [6]:
tickers = yf.Tickers(tickers)

In [7]:
df = tickers.download(start=start,end=today_date,interval='1D',)   #['Adj Close']
#df contains raw data downloaded from yfinance
df

[*********************100%%**********************]  504 of 504 completed


Price            Close                                                         \
Ticker               A    AAL        AAPL        ABBV        ABNB         ABT   
Date                                                                            
1990-01-02         NaN    NaN    0.263078         NaN         NaN    1.809340   
1990-01-03         NaN    NaN    0.264843         NaN         NaN    1.815872   
1990-01-04         NaN    NaN    0.265727         NaN         NaN    1.812607   
1990-01-05         NaN    NaN    0.266610         NaN         NaN    1.793011   
1990-01-08         NaN    NaN    0.268375         NaN         NaN    1.802201   
...                ...    ...         ...         ...         ...         ...   
2024-06-12  133.160004  11.50  213.070007  166.089996  149.320007  104.379997   
2024-06-13  130.570007  11.44  214.240005  166.559998  146.100006  103.449997   
2024-06-14  129.850006  11.28  212.490005  168.589996  145.970001  103.680000   
2024-06-17  131.830002  11.44  216.669998  169.679993  149.500000  103.449997   
2024-06-18  134.899994  11.28  214.289993  171.360001  147.289993  103.989998   

Price                                                       ...   Volume  \
Ticker            ACGL         ACN        ADBE         ADI  ...       WY   
Date                                                        ...            
1990-01-02         NaN         NaN    1.188340    1.007657  ...   442400   
1990-01-03         NaN         NaN    1.247023    0.955311  ...   289700   
1990-01-04         NaN         NaN    1.305707    0.942224  ...   324200   
1990-01-05         NaN         NaN    1.335048    0.942224  ...   214000   
1990-01-08         NaN         NaN    1.352693    0.942224  ...   275400   
...                ...         ...         ...         ...  ...      ...   
2024-06-12   99.160004  285.730011  459.869995  238.440002  ...  3590800   
2024-06-13   99.150002  282.320007  458.739990  234.029999  ...  3726400   
2024-06-14   97.260002  286.709991  525.309998  231.190002  ...  3378000   
2024-06-17   99.209999  285.529999  518.739990  232.389999  ...  3382500   
2024-06-18  100.800003  285.350006  522.250000  235.380005  ...  3076556   

Price                                                                      \
Ticker           WYNN      XEL       XOM        XYL        YUM        ZBH   
Date                                                                        
1990-01-02        NaN   247200   5326000        NaN        NaN        NaN   
1990-01-03        NaN   126800   4980400        NaN        NaN        NaN   
1990-01-04        NaN   204200   6013200        NaN        NaN        NaN   
1990-01-05        NaN   144800   3854800        NaN        NaN        NaN   
1990-01-08        NaN   189000   4302000        NaN        NaN        NaN   
...               ...      ...       ...        ...        ...        ...   
2024-06-12  2048700.0  3006100  13606800   669600.0  1279900.0  2996500.0   
2024-06-13  1627100.0  3339800  14466900   818900.0  1554700.0  1471000.0   
2024-06-14  2688600.0  2400600  13313400   883100.0  1418500.0  1611400.0   
2024-06-17  1832700.0  2255900  21147900  1175800.0  1327800.0  1579200.0   
2024-06-18  1602702.0  3163261  13180120   907218.0  2540831.0  1753140.0   

Price                                        
Ticker          ZBRA        ZTS       ^GSPC  
Date                                         
1990-01-02       NaN        NaN   162070000  
1990-01-03       NaN        NaN   192330000  
1990-01-04       NaN        NaN   177000000  
1990-01-05       NaN        NaN   158530000  
1990-01-08       NaN        NaN   140110000  
...              ...        ...         ...  
2024-06-12  191600.0  2068300.0  3962840000  
2024-06-13  303000.0  2298800.0  3530380000  
2024-06-14  218800.0  2225300.0  3438650000  
2024-06-17  306400.0  1798700.0  3447840000  
2024-06-18  435926.0  2273548.0  2461866000  

[8682 rows x 3528 columns]

In [8]:
list(df.columns.levels[0])

['Close', 'Dividends', 'High', 'Low', 'Open', 'Stock Splits', 'Volume']

In [9]:
df1=df['Close']

In [10]:
#df1 contains close data for all S&P 500 stocks and the index
df1

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,0.263078,NaN,NaN,1.809340,NaN,NaN,1.188340,1.007657,...,3.252945,NaN,4.055130,3.961588,NaN,NaN,NaN,NaN,NaN,359.690002
1990-01-03,NaN,NaN,0.264843,NaN,NaN,1.815872,NaN,NaN,1.247023,0.955311,...,3.181295,NaN,4.042615,3.921968,NaN,NaN,NaN,NaN,NaN,358.760010
1990-01-04,NaN,NaN,0.265727,NaN,NaN,1.812607,NaN,NaN,1.305707,0.942224,...,3.152636,NaN,3.942487,3.882354,NaN,NaN,NaN,NaN,NaN,355.670013
1990-01-05,NaN,NaN,0.266610,NaN,NaN,1.793011,NaN,NaN,1.335048,0.942224,...,3.138303,NaN,3.904940,3.862547,NaN,NaN,NaN,NaN,NaN,352.200012
1990-01-08,NaN,NaN,0.268375,NaN,NaN,1.802201,NaN,NaN,1.352693,0.942224,...,3.138303,NaN,3.867390,3.921968,NaN,NaN,NaN,NaN,NaN,353.790009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-12,133.160004,11.50,213.070007,166.089996,149.320007,104.379997,99.160004,285.730011,459.869995,238.440002,...,29.209999,90.830002,54.217667,110.930000,140.679993,137.300003,107.449997,304.559998,175.850006,5421.029785
2024-06-13,130.570007,11.44,214.240005,166.559998,146.100006,103.449997,99.150002,282.320007,458.739990,234.029999,...,29.020000,90.489998,53.792000,110.040001,140.669998,138.009995,106.970001,308.910004,171.320007,5433.740234
2024-06-14,129.850006,11.28,212.490005,168.589996,145.970001,103.680000,97.260002,286.709991,525.309998,231.190002,...,29.000000,88.800003,53.770000,109.110001,138.020004,136.789993,107.190002,301.369995,170.550003,5431.600098


In [11]:
#Calculate the daily returns
daily_return_df = np.log(df1/df1.shift(1))

In [12]:
#daily_return_df contains the daily return data for all S&P 500 stocks and the index
daily_return_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,0.006688,NaN,NaN,0.003604,NaN,NaN,0.048202,-0.053346,...,-0.022272,NaN,-0.003091,-0.010051,NaN,NaN,NaN,NaN,NaN,-0.002589
1990-01-04,NaN,NaN,0.003331,NaN,NaN,-0.001800,NaN,NaN,0.045986,-0.013794,...,-0.009049,NaN,-0.025080,-0.010152,NaN,NaN,NaN,NaN,NaN,-0.008650
1990-01-05,NaN,NaN,0.003316,NaN,NaN,-0.010870,NaN,NaN,0.022222,0.000000,...,-0.004557,NaN,-0.009569,-0.005115,NaN,NaN,NaN,NaN,NaN,-0.009804
1990-01-08,NaN,NaN,0.006601,NaN,NaN,0.005113,NaN,NaN,0.013130,0.000000,...,0.000000,NaN,-0.009662,0.015267,NaN,NaN,NaN,NaN,NaN,0.004504
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-12,0.007917,0.019316,0.028178,-0.009647,0.010570,-0.013607,-0.004227,-0.029280,-0.006113,0.009016,...,0.002399,-0.009969,-0.004918,-0.011116,0.015473,0.002114,-0.016430,0.010496,-0.015181,0.008468
2024-06-13,-0.019642,-0.005231,0.005476,0.002826,-0.021800,-0.008950,-0.000101,-0.012006,-0.002460,-0.018668,...,-0.006526,-0.003750,-0.007882,-0.008055,-0.000071,0.005158,-0.004477,0.014182,-0.026098,0.002342
2024-06-14,-0.005530,-0.014085,-0.008202,0.012114,-0.000890,0.002221,-0.019246,0.015430,0.135505,-0.012209,...,-0.000689,-0.018853,-0.000409,-0.008487,-0.019018,-0.008879,0.002055,-0.024711,-0.004505,-0.000394


In [13]:
#Calculate the monthly returns
M1_return_df = np.log(df1/df1.shift(period_1m))

In [14]:
#M1_return_df contains the monthly return data for all S&P 500 stocks and the index
M1_return_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-12,-0.104038,-0.261026,0.134370,0.029388,0.001273,-0.003729,0.012686,-0.073135,-0.049301,0.138724,...,-0.059475,-0.073298,-0.019169,-0.061022,-0.001572,0.011625,-0.107044,-0.036874,0.046203,0.037516
2024-06-13,-0.147223,-0.283521,0.133692,0.030293,-0.004098,-0.005399,0.012687,-0.083644,-0.036829,0.103116,...,-0.065360,-0.077559,-0.022203,-0.067040,-0.007290,0.008677,-0.114693,-0.038329,0.014404,0.035032
2024-06-14,-0.171092,-0.281011,0.113346,0.028885,0.001165,-0.008739,-0.006252,-0.073315,0.079118,0.073089,...,-0.071485,-0.092118,-0.026743,-0.083231,-0.038342,-0.006365,-0.121022,-0.072003,-0.015880,0.022990


In [15]:
#Calculate the M2 returns
M2_return_df = np.log(df1/df1.shift(period_2m))

In [16]:
#M2_return_df contains the 2 month return data for all S&P 500 stocks and the index
M2_return_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-12,-0.055292,-0.134075,0.188017,0.023207,-0.065137,-0.044318,0.097147,-0.098794,-0.030453,0.219187,...,-0.122491,-0.105334,0.029648,-0.081671,0.096212,0.001267,-0.147119,0.050091,0.161977,0.056466
2024-06-13,-0.071303,-0.123207,0.215599,0.029798,-0.062997,-0.051250,0.099494,-0.104024,-0.024462,0.210870,...,-0.112332,-0.091737,0.021766,-0.083978,0.102562,0.007369,-0.147733,0.087603,0.137280,0.070902
2024-06-14,-0.052140,-0.144223,0.226750,0.036546,-0.070677,-0.051963,0.068068,-0.090731,0.098109,0.193733,...,-0.090750,-0.109685,0.033467,-0.084159,0.077981,-0.003905,-0.129992,0.072050,0.111307,0.072566


In [17]:
#Calculate the M3 returns
M3_return_df = np.log(df1/df1.shift(period_3m))

In [18]:
#M3_return_df contains the 3 month return data for all S&P 500 stocks and the index
M3_return_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-12,-0.102821,-0.196710,0.219197,-0.070353,-0.098398,-0.135839,0.085875,-0.277885,-0.220901,0.186795,...,-0.161434,-0.116636,0.072259,0.012701,0.096914,-0.002788,-0.162049,0.068135,0.000689,0.048321
2024-06-13,-0.104648,-0.201227,0.213805,-0.074950,-0.130344,-0.133827,0.085445,-0.288544,-0.217942,0.189347,...,-0.159241,-0.107304,0.046327,-0.012912,0.097312,0.011080,-0.159710,0.091959,-0.011985,0.053538
2024-06-14,-0.125689,-0.213165,0.207802,-0.044344,-0.095765,-0.102928,0.056904,-0.263523,0.064575,0.173185,...,-0.161972,-0.107064,0.043800,-0.019603,0.083701,0.004537,-0.153191,0.052837,-0.008927,0.059648


In [19]:
#Calculate the M6 returns
M6_return_df = np.log(df1/df1.shift(period_6m))

In [20]:
#M6_return_df contains the 6 month return data for all S&P 500 stocks and the index
M6_return_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-12,0.037536,-0.190261,0.093647,0.134325,0.065593,0.012933,0.235676,-0.151872,-0.280513,0.265790,...,-0.039118,0.100773,-0.097621,0.128906,0.288197,0.109606,-0.083753,0.266843,-0.028328,0.167367
2024-06-13,0.029462,-0.184650,0.091738,0.128337,0.037804,-0.000428,0.223396,-0.170215,-0.284992,0.244413,...,-0.044996,0.077108,-0.095050,0.109435,0.284925,0.114200,-0.088315,0.268667,-0.069545,0.165622
2024-06-14,0.010113,-0.195823,0.096548,0.127406,0.021186,-0.014437,0.200248,-0.169736,-0.174083,0.204203,...,-0.052465,0.036984,-0.105912,0.100244,0.253575,0.087072,-0.092041,0.229159,-0.100037,0.161311


In [21]:
#Calculate the M12 returns
M12_return_df = np.log(df1/df1.shift(period_12m))

In [22]:
#M12_return_df contains the 12 month return data for all S&P 500 stocks and the index
M12_return_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-12,0.132859,-0.261026,0.177004,0.243864,0.241357,0.037077,0.331242,-0.059911,0.060460,0.310617,...,0.037858,-0.112717,-0.112499,0.070564,0.292341,0.052009,-0.184882,0.120076,0.042711,0.235434
2024-06-13,0.120481,-0.281544,0.190266,0.238821,0.225971,0.037526,0.330439,-0.081577,0.092237,0.278553,...,0.012845,-0.121516,-0.132111,0.040335,0.284865,0.060483,-0.210625,0.107013,0.055932,0.241595
2024-06-14,0.131382,-0.304822,0.166718,0.242618,0.234131,0.048441,0.314849,-0.054180,0.179421,0.259107,...,0.012834,-0.138430,-0.129874,0.034985,0.247616,0.045882,-0.225116,0.080610,0.047035,0.235031


In [78]:
#Calculate the 1 month forward returns
M1_forward_return_df = np.log(df1.shift(-period_1m)/df1)

In [79]:
#M1_forward_return_df contains the 1 month forward return data for all S&P 500 stocks and the index
M1_forward_return_df['NVDA'].tail(50)

Date
2024-04-09    0.057570
2024-04-10    0.019433
2024-04-11   -0.008178
2024-04-12    0.024785
2024-04-15    0.060405
2024-04-16    0.079308
2024-04-17    0.115873
2024-04-18    0.088208
2024-04-19    0.218197
2024-04-22    0.181948
2024-04-23    0.141486
2024-04-24    0.264475
2024-04-25    0.253457
2024-04-26    0.261009
2024-04-29    0.268838
2024-04-30    0.246005
2024-05-01    0.277804
2024-05-02    0.292715
2024-05-03    0.271088
2024-05-06    0.284312
2024-05-07    0.289828
2024-05-08    0.290487
2024-05-09    0.316509
2024-05-10    0.296675
2024-05-13    0.325761
2024-05-14    0.349848
2024-05-15    0.332000
2024-05-16    0.328020
2024-05-17    0.382663
2024-05-20         NaN
2024-05-21         NaN
2024-05-22         NaN
2024-05-23         NaN
2024-05-24         NaN
2024-05-28         NaN
2024-05-29         NaN
2024-05-30         NaN
2024-05-31         NaN
2024-06-03         NaN
2024-06-04         NaN
2024-06-05         NaN
2024-06-06         NaN
2024-06-07         NaN
2024-0

In [25]:
#Calculate the M1 standard deviation
M1_sd_df=daily_return_df.rolling(period_1m).std()

In [26]:
#M1_sd_df contains the 1 month standard deviation data for all S&P 500 stocks and the index
M1_sd_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-12,0.024828,0.033697,0.018280,0.013519,0.011054,0.011861,0.009879,0.013482,0.018703,0.025594,...,0.009961,0.010285,0.012196,0.012791,0.013577,0.011440,0.011348,0.012930,0.010190,0.005474
2024-06-13,0.024129,0.033057,0.018281,0.013522,0.011508,0.011916,0.009879,0.013599,0.018480,0.026050,...,0.009979,0.010244,0.012263,0.012838,0.013514,0.011361,0.011191,0.012836,0.011868,0.005432
2024-06-14,0.023427,0.033050,0.018498,0.013463,0.011429,0.011857,0.010789,0.014121,0.035054,0.026130,...,0.009805,0.010766,0.012207,0.012636,0.013788,0.011454,0.010881,0.013506,0.010418,0.004939


In [27]:
#Calculate the M2 standard deviation
M2_sd_df=daily_return_df.rolling(period_2m).std()

In [28]:
#M2_sd_df contains the 2 month standard deviation data for all S&P 500 stocks and the index
M2_sd_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-12,0.021175,0.031019,0.018134,0.013310,0.017425,0.010569,0.011851,0.011278,0.015523,0.021207,...,0.011277,0.015434,0.010774,0.011624,0.012631,0.011494,0.011304,0.016889,0.015958,0.006709
2024-06-13,0.021361,0.030954,0.017640,0.013297,0.017361,0.010638,0.011834,0.011357,0.015477,0.021394,...,0.011081,0.015257,0.010855,0.011648,0.012562,0.011520,0.011305,0.016549,0.016598,0.006365
2024-06-14,0.021061,0.030960,0.017344,0.013395,0.017312,0.010632,0.012187,0.011669,0.026040,0.021559,...,0.010642,0.015470,0.010672,0.011651,0.012979,0.011598,0.011170,0.016975,0.016386,0.006346


In [29]:
#Calculate the M3 standard deviation
M3_sd_df=daily_return_df.rolling(period_3m).std()

In [30]:
#M3_sd_df contains the 3 month standard deviation data for all S&P 500 stocks and the index
M3_sd_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-12,0.019088,0.028706,0.017429,0.014153,0.017261,0.011376,0.012854,0.016924,0.023840,0.020705,...,0.011864,0.015508,0.010578,0.010968,0.011056,0.010769,0.012140,0.017044,0.018196,0.006823
2024-06-13,0.019114,0.028705,0.017406,0.014120,0.017380,0.011353,0.012854,0.016946,0.023840,0.020660,...,0.011849,0.015444,0.010413,0.010788,0.011055,0.010730,0.012131,0.017066,0.018416,0.006809
2024-06-14,0.018993,0.028737,0.017453,0.014042,0.016846,0.010844,0.013071,0.017119,0.022956,0.020748,...,0.011837,0.015440,0.010412,0.010836,0.011323,0.010787,0.012142,0.017298,0.018400,0.006746


In [31]:
#Calculate the M6 standard deviation
M6_sd_df=daily_return_df.rolling(period_6m).std()

In [32]:
#M6_sd_df contains the 6 month standard deviation data for all S&P 500 stocks and the index
M6_sd_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-12,0.017553,0.028386,0.014906,0.011886,0.018978,0.010653,0.012872,0.014912,0.022202,0.018389,...,0.013853,0.016947,0.016038,0.011422,0.011340,0.010097,0.011704,0.023334,0.017628,0.006904
2024-06-13,0.017610,0.028376,0.014900,0.011867,0.019074,0.010676,0.012840,0.014927,0.022198,0.018480,...,0.013864,0.016866,0.016027,0.011412,0.011342,0.010104,0.011709,0.023340,0.017722,0.006900
2024-06-14,0.017576,0.028397,0.014871,0.011860,0.019024,0.010579,0.012972,0.014931,0.025213,0.018374,...,0.013850,0.016851,0.015996,0.011442,0.011460,0.010022,0.011697,0.023432,0.017567,0.006898


In [33]:
#Calculate the M12 standard deviation
M12_sd_df=daily_return_df.rolling(period_12m).std()

In [34]:
#M12_sd_df contains the 12 month standard deviation data for all S&P 500 stocks and the index
M12_sd_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-12,0.017121,0.024513,0.013558,0.011536,0.020966,0.011348,0.014035,0.013460,0.020295,0.016789,...,0.013443,0.017513,0.014105,0.012986,0.012249,0.009909,0.013416,0.025158,0.016733,0.007113
2024-06-13,0.017161,0.024493,0.013551,0.011528,0.021010,0.011347,0.014036,0.013466,0.020180,0.016817,...,0.013400,0.017511,0.014092,0.012924,0.012243,0.009912,0.013346,0.025117,0.016631,0.007107
2024-06-14,0.017132,0.024498,0.013532,0.011540,0.021001,0.011334,0.014091,0.013482,0.021673,0.016833,...,0.013400,0.017548,0.014091,0.012933,0.012260,0.009922,0.013303,0.025166,0.016631,0.007100


In [35]:
#Calculate the M12-M1 return
M12_1_return_df=M12_return_df-M1_return_df

In [36]:
#M12_1_return_df contains the 12 month return data minus 1 month return data for all S&P 500 stocks and the index
M12_1_return_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-12,0.236897,0.000000,0.042634,0.214477,0.240084,0.040806,0.318556,0.013224,0.109761,0.171892,...,0.097334,-0.039419,-0.093330,0.131586,0.293913,0.040384,-0.077838,0.156950,-0.003492,0.197917
2024-06-13,0.267704,0.001977,0.056575,0.208527,0.230069,0.042924,0.317752,0.002067,0.129066,0.175437,...,0.078205,-0.043957,-0.109908,0.107375,0.292155,0.051806,-0.095932,0.145342,0.041528,0.206563
2024-06-14,0.302474,-0.023811,0.053372,0.213733,0.232965,0.057179,0.321101,0.019136,0.100303,0.186018,...,0.084320,-0.046312,-0.103130,0.118216,0.285959,0.052248,-0.104093,0.152613,0.062916,0.212041


In [37]:
#Calculate the M12-M2 return
M12_2_return_df=M12_return_df-M2_return_df

In [38]:
#M12_2_return_df contains the 12 month return data minus 2 month return data for all S&P 500 stocks and the index
M12_2_return_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-12,0.188151,-0.126951,-0.011013,0.220658,0.306494,0.081395,0.234095,0.038883,0.090913,0.091430,...,0.160349,-0.007383,-0.142147,0.152235,0.196129,0.050742,-0.037762,0.069985,-0.119266,0.178968
2024-06-13,0.191784,-0.158337,-0.025332,0.209022,0.288968,0.088776,0.230945,0.022447,0.116699,0.067683,...,0.125177,-0.029778,-0.153878,0.124312,0.182303,0.053115,-0.062892,0.019410,-0.081348,0.170693
2024-06-14,0.183522,-0.160598,-0.060033,0.206072,0.304807,0.100404,0.246781,0.036551,0.081312,0.065374,...,0.103584,-0.028745,-0.163341,0.119144,0.169635,0.049787,-0.095124,0.008560,-0.064272,0.162465


In [39]:
#Calculate the M12-M3 return
M12_3_return_df=M12_return_df-M3_return_df

In [40]:
#M12_3_return_df contains the 12 month return data minus 3 month return data for all S&P 500 stocks and the index
M12_3_return_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-12,0.235680,-0.064315,-0.042193,0.314218,0.339755,0.172916,0.245367,0.217974,0.281361,0.123822,...,0.199292,0.003919,-0.184757,0.057863,0.195426,0.054797,-0.022832,0.051941,0.042022,0.187113
2024-06-13,0.225129,-0.080318,-0.023539,0.313771,0.356315,0.171352,0.244994,0.206967,0.310179,0.089206,...,0.172086,-0.014212,-0.178439,0.053246,0.187553,0.049403,-0.050915,0.015054,0.067917,0.188057
2024-06-14,0.257072,-0.091657,-0.041085,0.286961,0.329895,0.151368,0.257944,0.209343,0.114845,0.085922,...,0.174806,-0.031365,-0.173674,0.054588,0.163916,0.041345,-0.071924,0.027773,0.055963,0.175383


In [41]:
#Calculate the M12-M6 return
M12_6_return_df=M12_return_df-M6_return_df

In [42]:
#M12_6_return_df contains the 12 month return data minus 6 month return data for all S&P 500 stocks and the index
M12_6_return_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-12,0.095323,-0.070765,0.083357,0.109540,0.175764,0.024144,0.095566,0.091961,0.340973,0.044827,...,0.076976,-0.213490,-0.014878,-0.058342,0.004144,-0.057597,-0.101129,-0.146767,0.071039,0.068067
2024-06-13,0.091019,-0.096895,0.098528,0.110484,0.188167,0.037954,0.107043,0.088637,0.377229,0.034140,...,0.057841,-0.198624,-0.037061,-0.069100,-0.000060,-0.053717,-0.122310,-0.161655,0.125477,0.075973
2024-06-14,0.121269,-0.108998,0.070170,0.115211,0.212945,0.062877,0.114601,0.115556,0.353504,0.054904,...,0.065300,-0.175414,-0.023962,-0.065259,-0.005958,-0.041189,-0.133075,-0.148550,0.147072,0.073720


In [43]:
#Calculate dollar value by multiplying volume by closing position
dv_df=df['Close']*df['Volume']

In [44]:
#dv_df contains all the dollar value for all S&P 500 stocks and the index
dv_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,4.819545e+07,NaN,NaN,1.828603e+07,NaN,NaN,8.516119e+06,9.026593e+05,...,1.439103e+06,NaN,1.002428e+06,2.109942e+07,NaN,NaN,NaN,NaN,NaN,5.829496e+10
1990-01-03,NaN,NaN,5.508614e+07,NaN,NaN,1.110313e+07,NaN,NaN,1.685776e+07,6.247734e+05,...,9.216211e+05,NaN,5.126036e+05,1.953297e+07,NaN,NaN,NaN,NaN,NaN,6.900031e+10
1990-01-04,NaN,NaN,5.886215e+07,NaN,NaN,1.173892e+07,NaN,NaN,1.548465e+07,7.784654e+05,...,1.022085e+06,NaN,8.050558e+05,2.334537e+07,NaN,NaN,NaN,NaN,NaN,6.295359e+10
1990-01-05,NaN,NaN,3.287617e+07,NaN,NaN,6.064830e+06,NaN,NaN,8.821998e+06,2.747525e+05,...,6.715967e+05,NaN,5.654353e+05,1.488935e+07,NaN,NaN,NaN,NaN,NaN,5.583427e+10
1990-01-08,NaN,NaN,2.725963e+07,NaN,NaN,1.195095e+07,NaN,NaN,5.880426e+06,4.081714e+05,...,8.642885e+05,NaN,7.309368e+05,1.687231e+07,NaN,NaN,NaN,NaN,NaN,4.956952e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-12,3.195707e+08,4.688516e+08,4.221648e+10,7.848749e+08,5.345358e+08,4.769122e+08,1.279462e+08,1.071373e+09,1.677330e+09,5.996766e+08,...,1.048873e+08,1.860834e+08,1.629837e+08,1.509402e+09,9.419932e+07,1.757303e+08,3.219739e+08,5.835370e+07,3.637106e+08,2.148267e+13
2024-06-13,3.336586e+08,3.808376e+08,2.096611e+10,6.147396e+08,6.502035e+08,4.882323e+08,1.096004e+08,8.806126e+08,3.925897e+09,6.257494e+08,...,1.081401e+08,1.472363e+08,1.796545e+08,1.591938e+09,1.151947e+08,2.145641e+08,1.573529e+08,9.359973e+07,3.938304e+08,1.918317e+13
2024-06-14,2.809045e+08,4.768338e+08,1.490037e+10,7.380196e+08,4.020890e+08,3.818638e+08,1.593702e+08,8.504392e+08,7.799067e+09,5.207324e+08,...,9.796200e+07,2.387477e+08,1.290803e+08,1.452625e+09,1.218855e+08,1.940366e+08,1.727260e+08,6.593975e+07,3.795249e+08,1.867737e+13


In [45]:
#Calculate the daily dv change
dv_change_df = np.log(dv_df/dv_df.shift(1))
dv_change_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,0.133634,NaN,NaN,-0.498910,NaN,NaN,0.682851,-0.367956,...,-0.445641,NaN,-0.670678,-0.077142,NaN,NaN,NaN,NaN,NaN,0.168595
1990-01-04,NaN,NaN,0.066300,NaN,NaN,0.055682,NaN,NaN,-0.084962,0.219936,...,0.103465,NaN,0.451409,0.178295,NaN,NaN,NaN,NaN,NaN,-0.091713
1990-01-05,NaN,NaN,-0.582450,NaN,NaN,-0.660403,NaN,NaN,-0.562601,-1.041454,...,-0.419941,NaN,-0.353316,-0.449753,NaN,NaN,NaN,NaN,NaN,-0.120010
1990-01-08,NaN,NaN,-0.187341,NaN,NaN,0.678305,NaN,NaN,-0.405619,0.395817,...,0.252249,NaN,0.256731,0.125028,NaN,NaN,NaN,NaN,NaN,-0.119012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-12,0.150207,0.490944,0.167460,0.210929,0.096267,-0.514004,-0.174685,0.393183,0.285910,-0.199662,...,0.132483,-0.023929,-0.090164,0.076737,-0.064570,-0.516432,-0.029526,0.113193,0.099554,0.113415
2024-06-13,0.043140,-0.207913,-0.699903,-0.244325,0.195887,0.023459,-0.154768,-0.196079,0.850392,0.042559,...,0.030542,-0.234156,0.097385,0.053238,0.201210,0.199658,-0.715980,0.472505,0.079562,-0.113214
2024-06-14,-0.172103,0.224795,-0.341521,0.182772,-0.480612,-0.245727,0.374389,-0.034865,0.686409,-0.183714,...,-0.098848,0.483369,-0.330601,-0.091580,0.056458,-0.100562,0.093215,-0.350286,-0.037000,-0.026720


In [46]:
#Calculate the monthly dv change
M1_dv_change_df = np.log(dv_df.rolling(period_1m).sum()/dv_df.shift(period_1m).rolling(period_1m).sum())
M1_dv_change_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-12,0.429276,0.054309,0.120412,-0.041337,-0.211169,0.082160,0.238926,0.034222,0.197813,0.315205,...,0.022938,-0.256897,-0.211311,-0.255951,0.183510,-0.286881,0.021478,-0.384340,-0.545946,0.033396
2024-06-13,0.431460,0.042501,0.173065,-0.030750,-0.210047,0.045658,0.253578,0.070301,0.238392,0.312234,...,0.028844,-0.216762,-0.192648,-0.260272,0.176531,-0.287504,0.011117,-0.375166,-0.468914,0.007753
2024-06-14,0.426311,0.046873,0.174868,-0.041692,-0.282742,0.049423,0.260344,0.027013,0.391080,0.265175,...,-0.022977,-0.167173,-0.176523,-0.279624,0.143601,-0.296140,0.037834,-0.378503,-0.453084,-0.009800


In [47]:
#Calculate the 3M dv change
M2_dv_change_df = np.log(dv_df.rolling(period_2m).sum()/dv_df.shift(period_2m).rolling(period_2m).sum())
M2_dv_change_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-12,0.005809,0.066670,0.075839,-0.186690,-0.244995,-0.045225,0.029287,0.035850,-0.532232,0.062869,...,-0.117970,-0.042142,-0.450469,0.045795,0.394895,-0.033683,0.026953,-0.083360,-0.169115,-0.040701
2024-06-13,0.010164,0.076883,0.087234,-0.192671,-0.204988,-0.046723,0.018803,0.025801,-0.489532,0.094925,...,-0.109799,-0.046068,-0.454021,0.044840,0.400417,-0.065464,0.035492,-0.048394,-0.231832,-0.040167
2024-06-14,0.019727,0.087817,0.086789,-0.195218,-0.142445,-0.075524,0.026152,0.020899,-0.388034,0.107653,...,-0.100705,-0.025872,-0.460045,0.032408,0.392970,-0.063335,0.033910,-0.031709,-0.259799,-0.043169


In [48]:
#Calculate the 3M dv change
M3_dv_change_df = np.log(dv_df.rolling(period_3m).sum()/dv_df.shift(period_3m).rolling(period_3m).sum())
M3_dv_change_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-12,0.066689,-0.119920,0.080463,0.055476,-0.229116,0.171092,0.190716,0.311838,-0.110058,0.056742,...,-0.146564,-0.158047,-0.323338,0.053353,0.133192,0.126921,-0.250957,-0.085214,0.215773,0.015796
2024-06-13,0.060255,-0.114322,0.088208,0.037986,-0.231871,0.168223,0.186638,0.315951,-0.117154,0.044117,...,-0.151073,-0.168959,-0.374688,0.047189,0.125399,0.101251,-0.243461,-0.094391,0.168250,0.006774
2024-06-14,0.018748,-0.099657,0.069781,-0.056451,-0.255637,0.063329,0.153765,0.293573,-0.180055,-0.021358,...,-0.178721,-0.211632,-0.415134,0.006869,0.117666,0.058553,-0.264463,-0.120532,0.156246,-0.022339


In [49]:
#Calculate the 6M dv change
M6_dv_change_df = np.log(dv_df.rolling(period_6m).sum()/dv_df.shift(period_6m).rolling(period_6m).sum())
M6_dv_change_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-12,-0.045676,0.136696,0.107958,0.255561,-0.164324,0.150532,0.021106,0.329190,0.127664,0.118430,...,0.014521,-0.141721,0.121795,0.060925,0.056043,0.252531,-0.054644,-0.086156,0.434614,0.167001
2024-06-13,-0.032656,0.133962,0.115618,0.251211,-0.164761,0.156166,0.021472,0.337545,0.140190,0.124319,...,0.015995,-0.145113,0.119085,0.056681,0.049253,0.258049,-0.049626,-0.081766,0.442552,0.168881
2024-06-14,-0.029831,0.134322,0.115674,0.245981,-0.168983,0.153494,0.025653,0.339021,0.167287,0.121015,...,0.017739,-0.137989,0.120645,0.050637,0.052174,0.262019,-0.049783,-0.082336,0.432223,0.168701


In [50]:
#Calculate the 12M dv change
M12_dv_change_df = np.log(dv_df.rolling(period_12m).sum()/dv_df.shift(period_12m).rolling(period_12m).sum())
M12_dv_change_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-12,0.153554,0.072064,-0.033618,0.005894,0.050515,0.081732,0.120266,0.171561,0.387248,0.117210,...,-0.087551,-0.255562,0.280520,0.056494,0.109151,0.138581,0.217889,-0.065188,0.240713,0.077818
2024-06-13,0.152205,0.075203,-0.028469,0.010388,0.054205,0.085077,0.120231,0.174916,0.396039,0.117391,...,-0.074003,-0.255875,0.279950,0.061280,0.105433,0.139431,0.211450,-0.058650,0.241947,0.079794
2024-06-14,0.150512,0.078704,-0.025672,0.011583,0.055244,0.084441,0.119589,0.176103,0.403461,0.117215,...,-0.071991,-0.257715,0.270142,0.063357,0.098752,0.137964,0.207703,-0.059619,0.242020,0.080760


In [51]:
#Calculate the percentage of positive returns over the past 1 month
M1_positive_returns_df = (daily_return_df\
.rolling(window=period_1m, min_periods=period_1m)\
.agg(lambda x: (x > 0).sum()))/period_1m

M1_positive_returns_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-12,0.476190,0.380952,0.809524,0.52381,0.428571,0.428571,0.428571,0.380952,0.47619,0.571429,...,0.428571,0.380952,0.428571,0.380952,0.571429,0.476190,0.380952,0.428571,0.571429,0.666667
2024-06-13,0.428571,0.333333,0.809524,0.52381,0.428571,0.428571,0.428571,0.380952,0.47619,0.523810,...,0.428571,0.333333,0.428571,0.380952,0.523810,0.476190,0.333333,0.428571,0.523810,0.666667
2024-06-14,0.380952,0.333333,0.761905,0.52381,0.428571,0.428571,0.428571,0.380952,0.47619,0.476190,...,0.380952,0.333333,0.380952,0.333333,0.476190,0.428571,0.333333,0.380952,0.476190,0.619048


In [52]:
#Calculate the percentage of positive returns over past 2 months
M2_positive_returns_df = (daily_return_df\
.rolling(window=period_2m, min_periods=period_2m)\
.agg(lambda x: (x > 0).sum()))/period_2m

M2_positive_returns_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-12,0.523810,0.500000,0.666667,0.523810,0.500000,0.404762,0.50000,0.452381,0.47619,0.619048,...,0.404762,0.404762,0.571429,0.404762,0.642857,0.50000,0.357143,0.500000,0.547619,0.595238
2024-06-13,0.523810,0.500000,0.690476,0.547619,0.500000,0.404762,0.50000,0.452381,0.47619,0.619048,...,0.404762,0.404762,0.571429,0.404762,0.642857,0.52381,0.357143,0.523810,0.547619,0.619048
2024-06-14,0.523810,0.476190,0.690476,0.547619,0.476190,0.404762,0.47619,0.452381,0.47619,0.595238,...,0.404762,0.404762,0.571429,0.404762,0.619048,0.50000,0.380952,0.523810,0.523810,0.619048


In [53]:
#Calculate the percentage of positive returns over past 3 months
M3_positive_returns_df = (daily_return_df\
.rolling(window=period_3m, min_periods=period_3m)\
.agg(lambda x: (x > 0).sum()))/period_3m

M3_positive_returns_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-12,0.492063,0.507937,0.619048,0.523810,0.460317,0.396825,0.492063,0.428571,0.412698,0.571429,...,0.396825,0.428571,0.571429,0.492063,0.587302,0.492063,0.365079,0.492063,0.476190,0.555556
2024-06-13,0.492063,0.507937,0.619048,0.523810,0.444444,0.396825,0.476190,0.428571,0.412698,0.571429,...,0.396825,0.428571,0.555556,0.476190,0.587302,0.507937,0.365079,0.507937,0.476190,0.571429
2024-06-14,0.476190,0.507937,0.619048,0.539683,0.444444,0.412698,0.460317,0.444444,0.428571,0.555556,...,0.380952,0.428571,0.539683,0.476190,0.587302,0.507937,0.380952,0.492063,0.476190,0.571429


In [54]:
#Calculate the percentage of positive returns over past 6 months
M6_positive_returns_df = (daily_return_df\
.rolling(window=period_6m, min_periods=period_6m)\
.agg(lambda x: (x > 0).sum()))/period_6m

M6_positive_returns_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-12,0.523438,0.476562,0.500000,0.585938,0.492188,0.507812,0.539062,0.500000,0.437500,0.578125,...,0.515625,0.507812,0.523438,0.515625,0.593750,0.554688,0.445312,0.507812,0.523438,0.585938
2024-06-13,0.523438,0.476562,0.500000,0.585938,0.484375,0.500000,0.531250,0.492188,0.429688,0.570312,...,0.515625,0.500000,0.523438,0.507812,0.585938,0.554688,0.437500,0.507812,0.515625,0.585938
2024-06-14,0.515625,0.476562,0.500000,0.585938,0.476562,0.500000,0.523438,0.492188,0.429688,0.562500,...,0.507812,0.492188,0.515625,0.500000,0.578125,0.546875,0.437500,0.500000,0.507812,0.578125


In [55]:
#Calculate the percentage of positive returns over past 12 months
M12_positive_returns_df = (daily_return_df\
.rolling(window=period_12m, min_periods=period_12m)\
.agg(lambda x: (x > 0).sum()))/period_12m

M12_positive_returns_df

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS,^GSPC
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-12,0.527344,0.480469,0.535156,0.566406,0.492188,0.500000,0.542969,0.523438,0.496094,0.535156,...,0.507812,0.511719,0.500000,0.511719,0.558594,0.535156,0.453125,0.515625,0.503906,0.562500
2024-06-13,0.527344,0.476562,0.539062,0.566406,0.492188,0.500000,0.539062,0.519531,0.496094,0.531250,...,0.503906,0.507812,0.496094,0.507812,0.554688,0.539062,0.449219,0.515625,0.503906,0.566406
2024-06-14,0.527344,0.472656,0.535156,0.566406,0.492188,0.503906,0.539062,0.523438,0.496094,0.527344,...,0.503906,0.507812,0.496094,0.507812,0.550781,0.535156,0.449219,0.511719,0.500000,0.562500


In [56]:
#Convert all the dataframes into a single column

Close=df1.stack(future_stack=True).rename("Close")
Daily_Returns=daily_return_df.stack(future_stack=True).rename("Daily_Returns")
M1_return=M1_return_df.stack(future_stack=True).rename("M1_return")
M2_return=M2_return_df.stack(future_stack=True).rename("M2_return")
M3_return=M3_return_df.stack(future_stack=True).rename("M3_return")
M6_return=M6_return_df.stack(future_stack=True).rename("M6_return")
M12_return=M12_return_df.stack(future_stack=True).rename("M12_return")

M1_forward_return=M1_forward_return_df.stack(future_stack=True).rename("M1_forward_return")

M1_sd=M1_sd_df.stack(future_stack=True).rename("M1_sd")
M2_sd=M2_sd_df.stack(future_stack=True).rename("M2_sd")
M3_sd=M3_sd_df.stack(future_stack=True).rename("M3_sd")
M6_sd=M6_sd_df.stack(future_stack=True).rename("M6_sd")
M12_sd=M12_sd_df.stack(future_stack=True).rename("M12_sd")

M12_1_return=M12_1_return_df.stack(future_stack=True).rename("M12_1_return")
M12_2_return=M12_2_return_df.stack(future_stack=True).rename("M12_2_return")
M12_3_return=M12_3_return_df.stack(future_stack=True).rename("M12_3_return")
M12_6_return=M12_6_return_df.stack(future_stack=True).rename("M12_6_return")

dv_change=dv_change_df.stack(future_stack=True).rename("dv_change")
M1_dv_change=M1_dv_change_df.stack(future_stack=True).rename("M1_dv_change")
M2_dv_change=M2_dv_change_df.stack(future_stack=True).rename("M2_dv_change")
M3_dv_change=M3_dv_change_df.stack(future_stack=True).rename("M3_dv_change")
M6_dv_change=M6_dv_change_df.stack(future_stack=True).rename("M6_dv_change")
M12_dv_change=M12_dv_change_df.stack(future_stack=True).rename("M12_dv_change")

M1_positive_returns=M1_positive_returns_df.stack(future_stack=True).rename("M1_positive_returns")
M2_positive_returns=M2_positive_returns_df.stack(future_stack=True).rename("M2_positive_returns")
M3_positive_returns=M3_positive_returns_df.stack(future_stack=True).rename("M3_positive_returns")
M6_positive_returns=M6_positive_returns_df.stack(future_stack=True).rename("M6_positive_returns")
M12_positive_returns=M12_positive_returns_df.stack(future_stack=True).rename("M12_positive_returns")

In [57]:
#Combine all the dataframes into one

Combined_df = pd.merge(Daily_Returns, M1_return,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])
Combined_df = pd.merge(Combined_df, M2_return,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])
Combined_df = pd.merge(Combined_df, M3_return,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])
Combined_df = pd.merge(Combined_df, M6_return,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])
Combined_df = pd.merge(Combined_df, M12_return,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])

Combined_df = pd.merge(Combined_df, M1_forward_return,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])

Combined_df = pd.merge(Combined_df, M1_sd,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])
Combined_df = pd.merge(Combined_df, M2_sd,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])
Combined_df = pd.merge(Combined_df, M3_sd,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])
Combined_df = pd.merge(Combined_df, M6_sd,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])
Combined_df = pd.merge(Combined_df, M12_sd,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])

Combined_df = pd.merge(Combined_df, M12_1_return,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])
Combined_df = pd.merge(Combined_df, M12_2_return,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])
Combined_df = pd.merge(Combined_df, M12_3_return,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])
Combined_df = pd.merge(Combined_df, M12_6_return,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])

Combined_df = pd.merge(Combined_df, dv_change,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])
Combined_df = pd.merge(Combined_df, M1_dv_change,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])
Combined_df = pd.merge(Combined_df, M2_dv_change,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])
Combined_df = pd.merge(Combined_df, M3_dv_change,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])
Combined_df = pd.merge(Combined_df, M6_dv_change,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])
Combined_df = pd.merge(Combined_df, M12_dv_change,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])

Combined_df = pd.merge(Combined_df, M1_positive_returns,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])
Combined_df = pd.merge(Combined_df, M2_positive_returns,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])
Combined_df = pd.merge(Combined_df, M3_positive_returns,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])
Combined_df = pd.merge(Combined_df, M6_positive_returns,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])
Combined_df = pd.merge(Combined_df, M12_positive_returns,  how='left', left_on=['Date','Ticker'], right_on = ['Date','Ticker'])


In [58]:
8681*504

4375224

In [59]:
Combined_df

Daily_Returns  M1_return  M2_return  M3_return  M6_return  \
Date       Ticker                                                              
1990-01-02 A                 NaN        NaN        NaN        NaN        NaN   
           AAL               NaN        NaN        NaN        NaN        NaN   
           AAPL              NaN        NaN        NaN        NaN        NaN   
           ABBV              NaN        NaN        NaN        NaN        NaN   
           ABNB              NaN        NaN        NaN        NaN        NaN   
...                          ...        ...        ...        ...        ...   
2024-06-18 YUM         -0.011016  -0.040147  -0.013028  -0.008812   0.043612   
           ZBH         -0.003182  -0.119063  -0.111965  -0.169700  -0.096038   
           ZBRA        -0.023928  -0.073883   0.078241   0.034667   0.188894   
           ZTS         -0.007282  -0.025604   0.102640  -0.016417  -0.146374   
           ^GSPC        0.002518   0.034063   0.090728   0.057870   0.153317   

                   M12_return  M1_forward_return     M1_sd     M2_sd  \
Date       Ticker                                                      
1990-01-02 A              NaN                NaN       NaN       NaN   
           AAL            NaN                NaN       NaN       NaN   
           AAPL           NaN           0.091292       NaN       NaN   
           ABBV           NaN                NaN       NaN       NaN   
           ABNB           NaN                NaN       NaN       NaN   
...                       ...                ...       ...       ...   
2024-06-18 YUM       0.025138                NaN  0.010269  0.011721   
           ZBH      -0.231711                NaN  0.010776  0.010958   
           ZBRA      0.053069                NaN  0.014188  0.017116   
           ZTS       0.034224                NaN  0.010485  0.016342   
           ^GSPC     0.234759                NaN  0.005083  0.006260   

                      M3_sd  ...  M1_dv_change  M2_dv_change  M3_dv_change  \
Date       Ticker            ...                                             
1990-01-02 A            NaN  ...           NaN           NaN           NaN   
           AAL          NaN  ...           NaN           NaN           NaN   
           AAPL         NaN  ...           NaN           NaN           NaN   
           ABBV         NaN  ...           NaN           NaN           NaN   
           ABNB         NaN  ...           NaN           NaN           NaN   
...                     ...  ...           ...           ...           ...   
2024-06-18 YUM     0.010823  ...     -0.300944     -0.039964      0.060239   
           ZBH     0.012047  ...      0.093473      0.036748     -0.215208   
           ZBRA    0.017576  ...     -0.349126      0.075199     -0.051897   
           ZTS     0.018295  ...     -0.399152     -0.298923      0.130652   
           ^GSPC   0.006742  ...     -0.033315     -0.043384     -0.008514   

                   M6_dv_change  M12_dv_change  M1_positive_returns  \
Date       Ticker                                                     
1990-01-02 A                NaN            NaN                  NaN   
           AAL              NaN            NaN                  NaN   
           AAPL             NaN            NaN                  NaN   
           ABBV             NaN            NaN                  NaN   
           ABNB             NaN            NaN                  NaN   
...                         ...            ...                  ...   
2024-06-18 YUM         0.259352       0.138445             0.428571   
           ZBH        -0.064261       0.206802             0.285714   
           ZBRA       -0.070925      -0.054212             0.428571   
           ZTS         0.423093       0.244174             0.428571   
           ^GSPC       0.161407       0.080603             0.666667   

                   M2_positive_returns  M3_positive_returns  \
Date       Ticker                      

In [60]:
list(Combined_df.columns)

['Daily_Returns',
 'M1_return',
 'M2_return',
 'M3_return',
 'M6_return',
 'M12_return',
 'M1_forward_return',
 'M1_sd',
 'M2_sd',
 'M3_sd',
 'M6_sd',
 'M12_sd',
 'M12_1_return',
 'M12_2_return',
 'M12_3_return',
 'M12_6_return',
 'dv_change',
 'M1_dv_change',
 'M2_dv_change',
 'M3_dv_change',
 'M6_dv_change',
 'M12_dv_change',
 'M1_positive_returns',
 'M2_positive_returns',
 'M3_positive_returns',
 'M6_positive_returns',
 'M12_positive_returns']

In [61]:
#Obtain all the Market returns, forward returns and market standard deviation

Mkt_ret=Combined_df.xs("^GSPC", level="Ticker")['Daily_Returns'].rename("Mkt_ret")
Mkt_1m=Combined_df.xs("^GSPC", level="Ticker")['M1_return'].rename("Mkt_1m")
Mkt_2m=Combined_df.xs("^GSPC", level="Ticker")['M2_return'].rename("Mkt_2m")
Mkt_3m=Combined_df.xs("^GSPC", level="Ticker")['M3_return'].rename("Mkt_3m")
Mkt_6m=Combined_df.xs("^GSPC", level="Ticker")['M6_return'].rename("Mkt_6m")
Mkt_12m=Combined_df.xs("^GSPC", level="Ticker")['M12_return'].rename("Mkt_12m")

Mkt_M1_forward_return=Combined_df.xs("^GSPC", level="Ticker")['M1_forward_return'].rename("Mkt_M1_forward_return")

Mkt_std_1m=Combined_df.xs("^GSPC", level="Ticker")['M1_sd'].rename("Mkt_std_1m")
Mkt_std_2m=Combined_df.xs("^GSPC", level="Ticker")['M2_sd'].rename("Mkt_std_2m")
Mkt_std_3m=Combined_df.xs("^GSPC", level="Ticker")['M3_sd'].rename("Mkt_std_3m")
Mkt_std_6m=Combined_df.xs("^GSPC", level="Ticker")['M6_sd'].rename("Mkt_std_6m")
Mkt_std_12m=Combined_df.xs("^GSPC", level="Ticker")['M12_sd'].rename("Mkt_std_12m")

In [62]:
Mkt_M1_forward_return

Date
1990-01-02    0.088942
1990-01-03    0.087234
1990-01-04    0.072127
1990-01-05    0.059516
1990-01-08    0.070642
                ...   
2024-06-12         NaN
2024-06-13         NaN
2024-06-14         NaN
2024-06-17         NaN
2024-06-18         NaN
Name: Mkt_M1_forward_return, Length: 8682, dtype: float64

In [63]:
#Add the Market returns, forward returns and market standard deviation as columns into the new dataframe

Combined_df2 = pd.DataFrame(Mkt_ret).join(pd.DataFrame(Combined_df), rsuffix='Date')
Combined_df2 = pd.DataFrame(Mkt_1m).join(pd.DataFrame(Combined_df2), rsuffix='Date')
Combined_df2 = pd.DataFrame(Mkt_2m).join(pd.DataFrame(Combined_df2), rsuffix='Date')
Combined_df2 = pd.DataFrame(Mkt_3m).join(pd.DataFrame(Combined_df2), rsuffix='Date')
Combined_df2 = pd.DataFrame(Mkt_6m).join(pd.DataFrame(Combined_df2), rsuffix='Date')
Combined_df2 = pd.DataFrame(Mkt_12m).join(pd.DataFrame(Combined_df2), rsuffix='Date')

Combined_df2 = pd.DataFrame(Mkt_M1_forward_return).join(pd.DataFrame(Combined_df2), rsuffix='Date')

Combined_df2 = pd.DataFrame(Mkt_std_1m).join(pd.DataFrame(Combined_df2), rsuffix='Date')
Combined_df2 = pd.DataFrame(Mkt_std_2m).join(pd.DataFrame(Combined_df2), rsuffix='Date')
Combined_df2 = pd.DataFrame(Mkt_std_3m).join(pd.DataFrame(Combined_df2), rsuffix='Date')
Combined_df2 = pd.DataFrame(Mkt_std_6m).join(pd.DataFrame(Combined_df2), rsuffix='Date')
Combined_df2 = pd.DataFrame(Mkt_std_12m).join(pd.DataFrame(Combined_df2), rsuffix='Date')

In [64]:
Combined_df2

Mkt_std_12m  Mkt_std_6m  Mkt_std_3m  Mkt_std_2m  \
Date       Ticker                                                    
1990-01-02 A               NaN         NaN         NaN         NaN   
           AAL             NaN         NaN         NaN         NaN   
           AAPL            NaN         NaN         NaN         NaN   
           ABBV            NaN         NaN         NaN         NaN   
           ABNB            NaN         NaN         NaN         NaN   
...                        ...         ...         ...         ...   
2024-06-18 YUM        0.007094    0.006828    0.006742     0.00626   
           ZBH        0.007094    0.006828    0.006742     0.00626   
           ZBRA       0.007094    0.006828    0.006742     0.00626   
           ZTS        0.007094    0.006828    0.006742     0.00626   
           ^GSPC      0.007094    0.006828    0.006742     0.00626   

                   Mkt_std_1m  Mkt_M1_forward_return   Mkt_12m    Mkt_6m  \
Date       Ticker                                                          
1990-01-02 A              NaN               0.088942       NaN       NaN   
           AAL            NaN               0.088942       NaN       NaN   
           AAPL           NaN               0.088942       NaN       NaN   
           ABBV           NaN               0.088942       NaN       NaN   
           ABNB           NaN               0.088942       NaN       NaN   
...                       ...                    ...       ...       ...   
2024-06-18 YUM       0.005083                    NaN  0.234759  0.153317   
           ZBH       0.005083                    NaN  0.234759  0.153317   
           ZBRA      0.005083                    NaN  0.234759  0.153317   
           ZTS       0.005083                    NaN  0.234759  0.153317   
           ^GSPC     0.005083                    NaN  0.234759  0.153317   

                    Mkt_3m    Mkt_2m  ...  M1_dv_change  M2_dv_change  \
Date       Ticker                     ...                               
1990-01-02 A           NaN       NaN  ...           NaN           NaN   
           AAL         NaN       NaN  ...           NaN           NaN   
           AAPL        NaN       NaN  ...           NaN           NaN   
           ABBV        NaN       NaN  ...           NaN           NaN   
           ABNB        NaN       NaN  ...           NaN           NaN   
...                    ...       ...  ...           ...           ...   
2024-06-18 YUM     0.05787  0.090728  ...     -0.300944     -0.039964   
           ZBH     0.05787  0.090728  ...      0.093473      0.036748   
           ZBRA    0.05787  0.090728  ...     -0.349126      0.075199   
           ZTS     0.05787  0.090728  ...     -0.399152     -0.298923   
           ^GSPC   0.05787  0.090728  ...     -0.033315     -0.043384   

                   M3_dv_change  M6_dv_change  M12_dv_change  \
Date       Ticker                                              
1990-01-02 A                NaN           NaN            NaN   
           AAL              NaN           NaN            NaN   
           AAPL             NaN           NaN            NaN   
           ABBV             NaN           NaN            NaN   
           ABNB             NaN           NaN            NaN   
...                         ...           ...            ...   
2024-06-18 YUM         0.060239      0.259352       0.138445   
           ZBH        -0.215208     -0.064261       0.206802   
           ZBRA       -0.051897     -0.070925      -0.054212   
           ZTS         0.130652      0.423093       0.244174   
           ^GSPC      -0.008514      0.161407       0.080603   

                   M1_positive_returns  M2_positive_returns  \
Date       Ticker                                             
1990-01-02 A                       NaN                  NaN   
           AAL                     NaN                  NaN   
           AAPL                    NaN                  NaN   
           ABBV      

In [65]:
list(Combined_df2.columns)

['Mkt_std_12m',
 'Mkt_std_6m',
 'Mkt_std_3m',
 'Mkt_std_2m',
 'Mkt_std_1m',
 'Mkt_M1_forward_return',
 'Mkt_12m',
 'Mkt_6m',
 'Mkt_3m',
 'Mkt_2m',
 'Mkt_1m',
 'Mkt_ret',
 'Daily_Returns',
 'M1_return',
 'M2_return',
 'M3_return',
 'M6_return',
 'M12_return',
 'M1_forward_return',
 'M1_sd',
 'M2_sd',
 'M3_sd',
 'M6_sd',
 'M12_sd',
 'M12_1_return',
 'M12_2_return',
 'M12_3_return',
 'M12_6_return',
 'dv_change',
 'M1_dv_change',
 'M2_dv_change',
 'M3_dv_change',
 'M6_dv_change',
 'M12_dv_change',
 'M1_positive_returns',
 'M2_positive_returns',
 'M3_positive_returns',
 'M6_positive_returns',
 'M12_positive_returns']

In [66]:
#Drop ^GSPC (which is S&P 500) from the rows of the dataframe

Final_df=Combined_df2.drop('^GSPC', level='Ticker')

In [67]:
Final_df

Mkt_std_12m  Mkt_std_6m  Mkt_std_3m  Mkt_std_2m  \
Date       Ticker                                                    
1990-01-02 A               NaN         NaN         NaN         NaN   
           AAL             NaN         NaN         NaN         NaN   
           AAPL            NaN         NaN         NaN         NaN   
           ABBV            NaN         NaN         NaN         NaN   
           ABNB            NaN         NaN         NaN         NaN   
...                        ...         ...         ...         ...   
2024-06-18 XYL        0.007094    0.006828    0.006742     0.00626   
           YUM        0.007094    0.006828    0.006742     0.00626   
           ZBH        0.007094    0.006828    0.006742     0.00626   
           ZBRA       0.007094    0.006828    0.006742     0.00626   
           ZTS        0.007094    0.006828    0.006742     0.00626   

                   Mkt_std_1m  Mkt_M1_forward_return   Mkt_12m    Mkt_6m  \
Date       Ticker                                                          
1990-01-02 A              NaN               0.088942       NaN       NaN   
           AAL            NaN               0.088942       NaN       NaN   
           AAPL           NaN               0.088942       NaN       NaN   
           ABBV           NaN               0.088942       NaN       NaN   
           ABNB           NaN               0.088942       NaN       NaN   
...                       ...                    ...       ...       ...   
2024-06-18 XYL       0.005083                    NaN  0.234759  0.153317   
           YUM       0.005083                    NaN  0.234759  0.153317   
           ZBH       0.005083                    NaN  0.234759  0.153317   
           ZBRA      0.005083                    NaN  0.234759  0.153317   
           ZTS       0.005083                    NaN  0.234759  0.153317   

                    Mkt_3m    Mkt_2m  ...  M1_dv_change  M2_dv_change  \
Date       Ticker                     ...                               
1990-01-02 A           NaN       NaN  ...           NaN           NaN   
           AAL         NaN       NaN  ...           NaN           NaN   
           AAPL        NaN       NaN  ...           NaN           NaN   
           ABBV        NaN       NaN  ...           NaN           NaN   
           ABNB        NaN       NaN  ...           NaN           NaN   
...                    ...       ...  ...           ...           ...   
2024-06-18 XYL     0.05787  0.090728  ...      0.015905      0.375981   
           YUM     0.05787  0.090728  ...     -0.300944     -0.039964   
           ZBH     0.05787  0.090728  ...      0.093473      0.036748   
           ZBRA    0.05787  0.090728  ...     -0.349126      0.075199   
           ZTS     0.05787  0.090728  ...     -0.399152     -0.298923   

                   M3_dv_change  M6_dv_change  M12_dv_change  \
Date       Ticker                                              
1990-01-02 A                NaN           NaN            NaN   
           AAL              NaN           NaN            NaN   
           AAPL             NaN           NaN            NaN   
           ABBV             NaN           NaN            NaN   
           ABNB             NaN           NaN            NaN   
...                         ...           ...            ...   
2024-06-18 XYL         0.167143      0.053592       0.087442   
           YUM         0.060239      0.259352       0.138445   
           ZBH        -0.215208     -0.064261       0.206802   
           ZBRA       -0.051897     -0.070925      -0.054212   
           ZTS         0.130652      0.423093       0.244174   

                   M1_positive_returns  M2_positive_returns  \
Date       Ticker                                             
1990-01-02 A                       NaN                  NaN   
           AAL                     NaN                  NaN   
           AAPL                    NaN                  NaN   
           ABBV      

In [68]:
#Download data as csv if needed

#Final_df.to_csv(path_or_buf='/Users/bennyang/Downloads/S&P500 features.csv', index=True) 

In [69]:
Ticker_dict={"NVDA":"Nvidia",
             "META":"Meta",
             "TSLA":"Tesla",
             "AMZN":"Amazon",
             "GOOG":"Google",
             "MSFT":"Microsoft",
             "AAPL":"Apple",
             "GS":"Goldman",
             "JPM":"JPMorgan",
             "NFLX":"Netflix",
             "NKE":"Nike",
             "AAL":"AmericanAirlines",
             "LUV":"SouthwestAirlines",
             "GM":"GeneralMotors"}

In [70]:
#Download multiple tickers to csv
Ticker_list=['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BK', 'BBWI', 'BAX', 'BDX', 'BRK-B', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BX', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF-B', 'BLDR', 'BG', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'COR', 'CNC', 'CNP', 'CF', 'CHRW', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CPAY', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CCI', 'CSX', 'CMI', 'CVS', 'DHR', 'DRI', 'DVA', 'DAY', 'DECK', 'DE', 'DAL', 'DVN', 'DXCM', 'FANG', 'DLR', 'DFS', 'DG', 'DLTR', 'D', 'DPZ', 'DOV', 'DOW', 'DHI', 'DTE', 'DUK', 'DD', 'EMN', 'ETN', 'EBAY', 'ECL', 'EIX', 'EW', 'EA', 'ELV', 'LLY', 'EMR', 'ENPH', 'ETR', 'EOG', 'EPAM', 'EQT', 'EFX', 'EQIX', 'EQR', 'ESS', 'EL', 'ETSY', 'EG', 'EVRG', 'ES', 'EXC', 'EXPE', 'EXPD', 'EXR', 'XOM', 'FFIV', 'FDS', 'FICO', 'FAST', 'FRT', 'FDX', 'FIS', 'FITB', 'FSLR', 'FE', 'FI', 'FMC', 'F', 'FTNT', 'FTV', 'FOXA', 'FOX', 'BEN', 'FCX', 'GRMN', 'IT', 'GE', 'GEHC', 'GEV', 'GEN', 'GNRC', 'GD', 'GIS', 'GM', 'GPC', 'GILD', 'GPN', 'GL', 'GS', 'HAL', 'HIG', 'HAS', 'HCA', 'DOC', 'HSIC', 'HSY', 'HES', 'HPE', 'HLT', 'HOLX', 'HD', 'HON', 'HRL', 'HST', 'HWM', 'HPQ', 'HUBB', 'HUM', 'HBAN', 'HII', 'IBM', 'IEX', 'IDXX', 'ITW', 'ILMN', 'INCY', 'IR', 'PODD', 'INTC', 'ICE', 'IFF', 'IP', 'IPG', 'INTU', 'ISRG', 'IVZ', 'INVH', 'IQV', 'IRM', 'JBHT', 'JBL', 'JKHY', 'J', 'JNJ', 'JCI', 'JPM', 'JNPR', 'K', 'KVUE', 'KDP', 'KEY', 'KEYS', 'KMB', 'KIM', 'KMI', 'KLAC', 'KHC', 'KR', 'LHX', 'LH', 'LRCX', 'LW', 'LVS', 'LDOS', 'LEN', 'LIN', 'LYV', 'LKQ', 'LMT', 'L', 'LOW', 'LULU', 'LYB', 'MTB', 'MRO', 'MPC', 'MKTX', 'MAR', 'MMC', 'MLM', 'MAS', 'MA', 'MTCH', 'MKC', 'MCD', 'MCK', 'MDT', 'MRK', 'META', 'MET', 'MTD', 'MGM', 'MCHP', 'MU', 'MSFT', 'MAA', 'MRNA', 'MHK', 'MOH', 'TAP', 'MDLZ', 'MPWR', 'MNST', 'MCO', 'MS', 'MOS', 'MSI', 'MSCI', 'NDAQ', 'NTAP', 'NFLX', 'NEM', 'NWSA', 'NWS', 'NEE', 'NKE', 'NI', 'NDSN', 'NSC', 'NTRS', 'NOC', 'NCLH', 'NRG', 'NUE', 'NVDA', 'NVR', 'NXPI', 'ORLY', 'OXY', 'ODFL', 'OMC', 'ON', 'OKE', 'ORCL', 'OTIS', 'PCAR', 'PKG', 'PANW', 'PARA', 'PH', 'PAYX', 'PAYC', 'PYPL', 'PNR', 'PEP', 'PFE', 'PCG', 'PM', 'PSX', 'PNW', 'PNC', 'POOL', 'PPG', 'PPL', 'PFG', 'PG', 'PGR', 'PLD', 'PRU', 'PEG', 'PTC', 'PSA', 'PHM', 'QRVO', 'PWR', 'QCOM', 'DGX', 'RL', 'RJF', 'RTX', 'O', 'REG', 'REGN', 'RF', 'RSG', 'RMD', 'RVTY', 'RHI', 'ROK', 'ROL', 'ROP', 'ROST', 'RCL', 'SPGI', 'CRM', 'SBAC', 'SLB', 'STX', 'SRE', 'NOW', 'SHW', 'SPG', 'SWKS', 'SJM', 'SNA', 'SOLV', 'SO', 'LUV', 'SWK', 'SBUX', 'STT', 'STLD', 'STE', 'SYK', 'SMCI', 'SYF', 'SNPS', 'SYY', 'TMUS', 'TROW', 'TTWO', 'TPR', 'TRGP', 'TGT', 'TEL', 'TDY', 'TFX', 'TER', 'TSLA', 'TXN', 'TXT', 'TMO', 'TJX', 'TSCO', 'TT', 'TDG', 'TRV', 'TRMB', 'TFC', 'TYL', 'TSN', 'USB', 'UBER', 'UDR', 'ULTA', 'UNP', 'UAL', 'UPS', 'URI', 'UNH', 'UHS', 'VLO', 'VTR', 'VLTO', 'VRSN', 'VRSK', 'VZ', 'VRTX', 'VTRS', 'VICI', 'V', 'VST', 'VMC', 'WRB', 'GWW', 'WAB', 'WBA', 'WMT', 'DIS', 'WBD', 'WM', 'WAT', 'WEC', 'WFC', 'WELL', 'WST', 'WDC', 'WRK', 'WY', 'WMB', 'WTW', 'WYNN', 'XEL', 'XYL', 'YUM', 'ZBRA', 'ZBH', 'ZTS']

In [71]:
#create the individual dataframes inside a dictionary
dfs = {}

for ticker in Ticker_list:
    dfs[ticker] = Final_df.xs(ticker, level="Ticker")
    

In [72]:
#export the dataframes inside a dictionary as csv
#Remember to check file path
for ticker, data in dfs.items():
    data.to_csv(path_or_buf=f"/Users/bennyang/Downloads/Ticker Features/{ticker} features.csv",index=True)

In [73]:
#Download single ticker to csv

Apple=Final_df.xs("AAPL", level="Ticker")

In [74]:
Apple

,Mkt_std_12m,Mkt_std_6m,Mkt_std_3m,Mkt_std_2m,Mkt_std_1m,Mkt_M1_forward_return,Mkt_12m,Mkt_6m,Mkt_3m,Mkt_2m,...,M1_dv_change,M2_dv_change,M3_dv_change,M6_dv_change,M12_dv_change,M1_positive_returns,M2_positive_returns,M3_positive_returns,M6_positive_returns,M12_positive_returns
Date,,,,,,,,,,,,,,,,,,,,,
1990-01-02,NaN,NaN,NaN,NaN,NaN,0.088942,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,NaN,NaN,NaN,0.087234,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-04,NaN,NaN,NaN,NaN,NaN,0.072127,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-05,NaN,NaN,NaN,NaN,NaN,0.059516,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-08,NaN,NaN,NaN,NaN,NaN,0.070642,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-12,0.007113,0.006904,0.006823,0.006709,0.005474,NaN,0.235434,0.167367,0.048321,0.056466,...,0.120412,0.075839,0.080463,0.107958,-0.033618,0.809524,0.666667,0.619048,0.500000,0.535156
2024-06-13,0.007107,0.006900,0.006809,0.006365,0.005432,NaN,0.241595,0.165622,0.053538,0.070902,...,0.173065,0.087234,0.088208,0.115618,-0.028469,0.809524,0.690476,0.619048,0.500000,0.539062
2024-06-14,0.007100,0.006898,0.006746,0.006346,0.004939,NaN,0.235031,0.161311,0.059648,0.072566,...,0.174868,0.086789,0.069781,0.115674,-0.025672,0.761905,0.690476,0.619048,0.500000,0.535156


In [75]:
#Remember to check file path
Apple.to_csv(path_or_buf='/Users/bennyang/Downloads/Apple features.csv', index=True) 